In [ ]:
import os
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import json

def process_node(state: MessagesState):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    response = llm.invoke(state["messages"])
    return {
        "messages": [response]
    }

workflow = StateGraph(MessagesState)
workflow.add_node("process", process_node)

workflow.add_edge(START, "process")
workflow.add_edge("process", END)

graph = workflow.compile()

ouput_str= []
for output in graph.stream({"messages": [HumanMessage(content="who is your daddy")]}, stream_mode="updates"):
    ouput_str.append(output)
    print(output)


with open("/home/user/test_output.txt", "w") as write:
    write.write("\n".join(str(output) for output in ouput_str))

In [ ]:
py_code = """
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# need to induce this import
import json

def process_node(state: MessagesState):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    response = llm.invoke(state["messages"])
    return {
        "messages": [response]
    }

workflow = StateGraph(MessagesState)
workflow.add_node("process", process_node)

workflow.add_edge(START, "process")
workflow.add_edge("process", END)

# this needs to be consistent
graph = workflow.compile()

# consistent
ouput_str= []
for output in graph.stream({"messages": [HumanMessage(content="who is your daddy")]}, stream_mode="updates"):
    ouput_str.append(output)
    print(output)


# consistent
with open("/home/user/test_output.txt", "w") as write:
    write.write("\\n".join(str(output) for output in ouput_str))
""" 

In [41]:
from openevals.code.e2b.execution import create_e2b_execution_evaluator
from dotenv import load_dotenv
from e2b_code_interpreter import Sandbox
import os

load_dotenv()
sandbox = Sandbox(envs= {"OPENAI_API_KEY" : os.environ["OPENAI_API_KEY"]})
evaluator = create_e2b_execution_evaluator(
    sandbox=sandbox,
)

In [42]:
eval_result = evaluator(outputs=py_code)

In [43]:
eval_result

{'key': 'execution_succeeded',
 'score': True,
 'comment': None,
 'metadata': None}

In [44]:
files = sandbox.files.read("/home/user/test_output.txt")

In [45]:
files

'{\'process\': {\'messages\': [AIMessage(content=\'I don\\\'t have a "daddy" in the traditional sense, but I was created by OpenAI, a research organization focused on artificial intelligence. If you have any questions about AI or anything else, feel free to ask!\', additional_kwargs={\'refusal\': None}, response_metadata={\'token_usage\': {\'completion_tokens\': 46, \'prompt_tokens\': 11, \'total_tokens\': 57, \'completion_tokens_details\': {\'accepted_prediction_tokens\': 0, \'audio_tokens\': 0, \'reasoning_tokens\': 0, \'rejected_prediction_tokens\': 0}, \'prompt_tokens_details\': {\'audio_tokens\': 0, \'cached_tokens\': 0}}, \'model_name\': \'gpt-4o-mini-2024-07-18\', \'system_fingerprint\': \'fp_e2f22fdd96\', \'id\': \'chatcmpl-BVLcVmjEMhRFZotkYywI5BP49eWGf\', \'service_tier\': \'default\', \'finish_reason\': \'stop\', \'logprobs\': None}, id=\'run--1f0673bf-dc1e-4004-b4ee-42817baa5d7e-0\', usage_metadata={\'input_tokens\': 11, \'output_tokens\': 46, \'total_tokens\': 57, \'input_t

In [34]:
#TODO setup .env files

#TODO setup the string so that there are no issues like \n \\n stuff